In [2]:
from tensorflow.keras.layers import Input, Flatten, Dense, Conv2D, MaxPooling2D
from tensorflow.keras.applications import mobilenet
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
import numpy as np

In [7]:
import tensorflow as tf
from tensorflow.keras.datasets.cifar10 import load_data
(X_train, y_train), (X_test, y_test) = load_data()
print(X_train.shape)
print(X_test.shape)

y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

X_train_resized = tf.image.resize(X_train, (224, 224))
X_test_resized = tf.image.resize(X_test, (224, 224))
print(X_train_resized.shape)
print(X_test_resized.shape)

170498071/170498071 [==============================] - 2s 0us/step
(50000, 32, 32, 3)
(10000, 32, 32, 3)
(50000, 224, 224, 3)
(10000, 224, 224, 3)


## Building a CNN

In [5]:
mobileNet = mobilenet.MobileNet(include_top=False, input_shape=(224, 224, 3))

input = mobileNet.inputs
x = mobileNet.output
x = Flatten(name='Flatten-Layer')(x)
x = Dense (128, activation='relu', name='Hidden-3')(x)
x = Dense(64, activation='relu', name='Hidden-4')(x)
x = Dense(32, activation='relu', name='Hidden-5')(x)
x = Dense(26, activation='relu', name='Hidden-6')(x)
output = Dense(10, activation='softmax', name='Output-Layer')(x)

model = Model(input, output, name='FCNN')
model.summary()

17225924/17225924 [==============================] - 0s 0us/step
Model: "FCNN"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizati  (None, 112, 112, 32)      128       
 on)                                                             
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D  (None, 112, 112, 32)      288       
 )                                                               
                                                               

## Train and Evaluate

In [3]:
def train_and_evaluate(model, name, validation_split=0.0):
  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
  print(f"Training {name}...")
  print(X_train_resized.shape, y_train.shape)
  model.fit(X_train_resized, y_train, epochs=10, batch_size=256, validation_split=validation_split)
  print(f"Trained {name}.")

  test_loss, test_accuracy = model.evaluate(X_test_resized, y_test)
  print(f"Loss: {test_loss}")
  print(f"Accuracy: {test_accuracy}")

  return model.predict(X_test_resized)

## Training CNN without validation set.

In [8]:
cnn_pred_y = train_and_evaluate(model, "CNN")

Training CNN...
(50000, 224, 224, 3) (50000, 10)
Epoch 1/10


KeyboardInterrupt: 

In [1]:
print("TestY \t\t PredY")
print("===== \t\t =====")
for i in range(10):
  print(np.argmax(y_test[i]), " \t\t ", np.argmax(cnn_pred_y[i]))

TestY 		 PredY
===== 		 =====


NameError: name 'np' is not defined

## Training CNN with validation set.

In [ ]:
cnn_pred_y = train_and_evaluate(cnn, "CNN", 0.2)

Training CNN...
(60000, 28, 28, 1) (60000, 10)
Epoch 1/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 50s 255ms/step - accuracy: 0.9237 - loss: 0.2081 - val_accuracy: 0.9273 - val_loss: 0.1963
Epoch 2/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 42s 221ms/step - accuracy: 0.9331 - loss: 0.1810 - val_accuracy: 0.9250 - val_loss: 0.2016
Epoch 3/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 81s 214ms/step - accuracy: 0.9404 - loss: 0.1639 - val_accuracy: 0.9190 - val_loss: 0.2246
Epoch 4/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 43s 223ms/step - accuracy: 0.9448 - loss: 0.1460 - val_accuracy: 0.9236 - val_loss: 0.2145
Epoch 5/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 80s 213ms/step - accuracy: 0.9514 - loss: 0.1330 - val_accuracy: 0.9167 - val_loss: 0.2377
Epoch 6/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 41s 216ms/step - accuracy: 0.9529 - loss: 0.1284 - val_accuracy: 0.9223 - val_loss: 0.2271
Epoch 7/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 42s 223ms/step - accuracy: 0.9518 - loss: 0.1292 - val_accuracy: 0.9210 - val_loss: 0.2429
Epoch 8/10
188/188 ━━━━━━━━━━━━━━━━━

In [ ]:
print("TestY \t\t PredY")
print("===== \t\t =====")
for i in range(10):
  print(np.argmax(y_test[i]), " \t\t ", np.argmax(cnn_pred_y[i]))

TestY 		 PredY
===== 		 =====
9  		  9
2  		  2
1  		  1
1  		  1
6  		  6
1  		  1
4  		  4
6  		  6
5  		  5
7  		  7
